# Prize-Collecting Steiner Tree (PCSTP)

## Libs Importing

In [1]:
import sys
import os
import time
import networkx as nx

sys.path.insert(1, os.path.realpath(os.path.pardir))

In [2]:
import multiprocessing

NUM_PROCESSES = multiprocessing.cpu_count()
print("Number of cpu : ", NUM_PROCESSES)

Number of cpu :  12


In [3]:
from pcstp.instances.generator import generate_random_steiner
from pcstp.instances.reader import SteinlibReader, DatReader

from pcstp.steinertree import SteinerTreeProblem
from pcstp.solver.base import computes_steiner_cost
from pcstp.solver.aco import AntColony
from pcstp.solver.greedy_h1 import GreedyH1

from pcstp.utils.graph import preprocessing
from pcstp.utils.draw import draw_steiner_graph

## Experiments

In [4]:
SEED = 100

In [5]:
import glob

INSTANCES_PATH_PREFIX = '../data/instances/benchmark/PCSPG-JMP'
NUM_EXPERIMENTS_PER_INSTANCE = 10

all_files = glob.glob(os.path.join(INSTANCES_PATH_PREFIX, '*'))

files = all_files

networkx_history = []

for filename in files:
    if filename.endswith('.stp'):
        stp_reader = SteinlibReader()
    else:
        stp_reader = DatReader()

    print(f"Reading: {filename}")
    stp = stp_reader.parser(filename=filename)
    G, terminals = preprocessing(stp.graph, stp.terminals)
    stp_preprocessed = SteinerTreeProblem(graph=G, terminals=terminals)
    
    def run_experiment(experiment: int):
        start_time = time.time()
        nx_steiner_tree = nx.algorithms.approximation.steiner_tree(
            stp_preprocessed.graph,
            stp_preprocessed.terminals,
            weight='cost'
        )

        networkx_duration = time.time() - start_time
        networkx_cost = computes_steiner_cost(stp.graph, nx_steiner_tree, stp.terminals)

        history = {
            "filename": filename,
            "experiment": experiment,
            "num_nodes": stp.num_nodes,
            "num_edges": stp.num_edges,
            "num_nodes_after_preprocessing": len(stp_preprocessed.graph.nodes),
            "num_edges_after_preprocessing": len(stp_preprocessed.graph.edges),
            "terminals": stp.num_terminals,
            "steiner_cost": networkx_cost,
            "duration": networkx_duration
        }
        return history

    experiments = range(1, NUM_EXPERIMENTS_PER_INSTANCE+1)

    with multiprocessing.Pool(processes=NUM_PROCESSES) as p:
        experiments_results = p.map(run_experiment, experiments)
    
    networkx_history.extend(experiments_results)

Reading: ../data/instances/benchmark/PCSPG-JMP/K100.5.stp
Reading: ../data/instances/benchmark/PCSPG-JMP/K100.1.stp
Reading: ../data/instances/benchmark/PCSPG-JMP/K100.3.stp
Reading: ../data/instances/benchmark/PCSPG-JMP/K200.stp
Reading: ../data/instances/benchmark/PCSPG-JMP/K400.4.stp


In [6]:
import pandas as pd

df_score_networkx = pd.DataFrame.from_dict(networkx_history)

In [7]:
df_score_networkx.groupby('filename')[['duration', 'steiner_cost']].describe()

duration                      \
                                                    count      mean       std   
filename                                                                        
../data/instances/benchmark/PCSPG-JMP/K100.1.stp     10.0  0.064955  0.003679   
../data/instances/benchmark/PCSPG-JMP/K100.3.stp     10.0  0.076225  0.003226   
../data/instances/benchmark/PCSPG-JMP/K100.5.stp     10.0  0.221893  0.008225   
../data/instances/benchmark/PCSPG-JMP/K200.stp       10.0  0.257868  0.008213   
../data/instances/benchmark/PCSPG-JMP/K400.4.stp     10.0  1.993021  0.063131   

                                                                      \
                                                       min       25%   
filename                                                               
../data/instances/benchmark/PCSPG-JMP/K100.1.stp  0.058443  0.063878   
../data/instances/benchmark/PCSPG-JMP/K100.3.stp  0.071538  0.075403   
../data/instances/benchmark/PCSPG-JMP/K100.5.stp  0.207741  0.216346   
../data/instances/benchmark/PCSPG-JMP/K200.stp    0.241132  0.254739   
../data/instances/benchmark/PCSPG-JMP/K400.4.stp  1.904132  1.941393   

                                                                      \
                                                       50%       75%   
filename                                                               
../data/instances/benchmark/PCSPG-JMP/K100.1.stp  0.066134  0.067173   
../data/instances/benchmark/PCSPG-JMP/K100.3.stp  0.075934  0.077592   
../data/instances/benchmark/PCSPG-JMP/K100.5.stp  0.224039  0.228448   
../data/instances/benchmark/PCSPG-JMP/K200.stp    0.260240  0.263242   
../data/instances/benchmark/PCSPG-JMP/K400.4.stp  2.004519  2.048746   

                                                           steiner_cost  \
                                                       max        count   
filename                                                                  
../data/instances/benchmark/PCSPG-JMP/K100.1.stp  0.069469         10.0   
../data/instances/benchmark/PCSPG-JMP/K100.3.stp  0.081210         10.0   
../data/instances/benchmark/PCSPG-JMP/K100.5.stp  0.231742         10.0   
../data/instances/benchmark/PCSPG-JMP/K200.stp    0.268274         10.0   
../data/instances/benchmark/PCSPG-JMP/K400.4.stp  2.069589         10.0   

                                                                           \
                                                      mean  std       min   
filename                                                                    
../data/instances/benchmark/PCSPG-JMP/K100.1.stp  259638.0  0.0  259638.0   
../data/instances/benchmark/PCSPG-JMP/K100.3.stp  212452.0  0.0  212452.0   
../data/instances/benchmark/PCSPG-JMP/K100.5.stp  270145.0  0.0  270145.0   
../data/instances/benchmark/PCSPG-JMP/K200.stp    489852.0  0.0  489852.0   
../data/instances/benchmark/PCSPG-JMP/K400.4.stp  528402.0  0.0  528402.0   

                                                                      \
                                                       25%       50%   
filename                                                               
../data/instances/benchmark/PCSPG-JMP/K100.1.stp  259638.0  259638.0   
../data/instances/benchmark/PCSPG-JMP/K100.3.stp  212452.0  212452.0   
../data/instances/benchmark/PCSPG-JMP/K100.5.stp  270145.0  270145.0   
../data/instances/benchmark/PCSPG-JMP/K200.stp    489852.0  489852.0   
../data/instances/benchmark/PCSPG-JMP/K400.4.stp  528402.0  528402.0   

                                                                      
                                                       75%       max  
filename                                                              
../data/instances/benchmark/PCSPG-JMP/K100.1.stp  259638.0  259638.0  
../data/instances/benchmark/PCSPG-JMP/K100.3.stp  212452.0  212452.0  
../data/instances/benchmark/PCSPG-JMP/K100.5.stp  270145.0  270145.0  
../data/instances/benchma


# Greedy

In [8]:
import glob
import random
import numpy as np

INSTANCES_PATH_PREFIX = '../data/instances/benchmark/E'
NUM_EXPERIMENTS_PER_INSTANCE = 5

all_files = glob.glob(os.path.join(INSTANCES_PATH_PREFIX, '*'))

files = all_files

greedy_history = []

for filename in files:
    if filename.endswith('.stp'):
        stp_reader = SteinlibReader()
    else:
        stp_reader = DatReader()

    print(f"Reading: {filename}")
    stp = stp_reader.parser(filename=filename)
    G, terminals = preprocessing(stp.graph, stp.terminals)
    stp_preprocessed = SteinerTreeProblem(graph=G, terminals=terminals)

    def run_experiment(experiment: int):
        if SEED:
            np.random.seed(SEED*experiment)
            random.seed(SEED*experiment)
        solver = GreedyH1(stp_preprocessed.graph, list(stp_preprocessed.terminals), log_level='info')
        steiner_tree, greedy_cost = solver.solve()
        print(f'Cost : {greedy_cost}')

        history = {
            "filename": filename,
            "experiment": experiment,
            "num_nodes": stp.num_nodes,
            "num_edges": stp.num_edges,
            "num_nodes_after_preprocessing": len(stp_preprocessed.graph.nodes),
            "num_edges_after_preprocessing": len(stp_preprocessed.graph.edges),
            "terminals": stp.num_terminals,
            "steiner_cost": greedy_cost,
            "duration": solver._duration
        }
        return history

    experiments = range(1, NUM_EXPERIMENTS_PER_INSTANCE+1)

    with multiprocessing.Pool(processes=NUM_PROCESSES) as p:
        experiments_results = p.map(run_experiment, experiments)

    greedy_history.extend(experiments_results)


Reading: ../data/instances/benchmark/E/e01.stp-A
Cost : 168.0Cost : 168.0
Cost : 168.0

Cost : 168.0
Cost : 168.0
Reading: ../data/instances/benchmark/E/e02.stp-A
Cost : 527.0
Cost : 527.0
Cost : 527.0
Cost : 527.0
Cost : 527.0
Reading: ../data/instances/benchmark/E/e04.stp-A


In [ ]:
import pandas as pd

df_score_greedy = pd.DataFrame.from_dict(greedy_history)

## Solution obtained with Ant Colony Optimization

In [ ]:
import glob

INSTANCES_PATH_PREFIX = '../data/instances/benchmark/E'
NUM_EXPERIMENTS_PER_INSTANCE = 5

all_files = glob.glob(os.path.join(INSTANCES_PATH_PREFIX, '*'))

files = all_files

aco_history = []

for filename in files:
    if filename.endswith('.stp'):
        stp_reader = SteinlibReader()
    else:
        stp_reader = DatReader()

    print(f"Reading: {filename}")
    stp = stp_reader.parser(filename=filename)
    G, terminals = preprocessing(stp.graph, stp.terminals)
    stp_preprocessed = SteinerTreeProblem(graph=G, terminals=terminals)

    def run_experiment(experiment: int):
        aco_params = dict(
            iterations=1,
            num_ants=1000,
            evaporation_rate=0.5,
            alpha=1.0,
            beta=3.0,
            # beta_evaporation_rate=0.2,
            initial_pheromone=0.1,
            pheromone_amount=2.0,
            pheromone_deposit_strategy='traditional',
            pheromone_initialization_strategy='heuristic',
            choose_best=0.2,
            log_level='info',
            early_stopping=30,
            normalize_distance_prize=False,
            allow_edge_perturbation=False,
            seed=SEED * experiment
        )
        solver = AntColony(
            graph=stp_preprocessed.graph,
            terminals=stp_preprocessed.terminals,
            **aco_params
        )
        steiner_tree, steiner_cost = solver.solve()

        history = {
            "filename": filename,
            "experiment": experiment,
            "num_nodes": stp.num_nodes,
            "num_edges": stp.num_edges,
            "num_nodes_after_preprocessing": len(stp_preprocessed.graph.nodes),
            "num_edges_after_preprocessing": len(stp_preprocessed.graph.edges),
            "terminals": stp.num_terminals,
            "steiner_cost": steiner_cost,
            "duration": solver._duration
        }
        history.update(aco_params)
        return history

    experiments = range(1, NUM_EXPERIMENTS_PER_INSTANCE+1)

    with multiprocessing.Pool(processes=NUM_PROCESSES) as p:
        experiments_results = p.map(run_experiment, experiments)
    
    aco_history.extend(experiments_results)

In [ ]:
import pandas as pd

df_score_aco = pd.DataFrame.from_dict(networkx_history)
df_score_aco.to_csv(os.path.join(INSTANCES_PATH_PREFIX, 'ACO.csv'))